In [4]:
from keras.applications.mobilenet_v2 import MobileNetV2
import numpy as np
from imageio import imread
from keras.applications.mobilenet_v2 import preprocess_input
from skimage.transform import resize

In [5]:
model = MobileNetV2(weights="imagenet")
model.summary()

14536120/14536120 [==============================] - 0s 0us/step
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
              

In [6]:
data = np.empty((24,224,224,3))
# print(data)
for i in range (12):
    im = imread('cats/f{:02d}.jpg'.format(i+1))
    im = preprocess_input(im)
    im = resize(im, output_shape=(224,224))
    data[i] = im
for i in range (12):
    im = imread('cats/m{:02d}.jpg'.format(i+1))
    im = preprocess_input(im)
    im = resize(im, output_shape=(224,224))
    data[i + 12] = im

labels = np.empty(24, dtype = int)
labels[:12] = 0 
labels[12:] = 1



C:\Users\mhalama\AppData\Local\Temp\ipykernel_14556\1328522048.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  im = imread('cats/f{:02d}.jpg'.format(i+1))
C:\Users\mhalama\AppData\Local\Temp\ipykernel_14556\1328522048.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  im = imread('cats/m{:02d}.jpg'.format(i+1))


In [7]:

from keras.applications.mobilenet_v2 import decode_predictions

predictions = model.predict(data)
for decoded_prediction in decode_predictions(predictions, top=1):
    for name, desc, score in decoded_prediction:
        print(' - {} ({:.2f}%)'.format(desc, 100 * score))
        


1/1 [==============================] - 2s 2s/step
 - Egyptian_cat (50.55%)
 - tabby (64.92%)
 - tabby (46.92%)
 - Egyptian_cat (44.26%)
 - Persian_cat (92.98%)
 - lynx (32.72%)
 - refrigerator (22.00%)
 - Egyptian_cat (28.10%)
 - malamute (22.11%)
 - tabby (15.47%)
 - guillotine (25.38%)
 - lynx (32.76%)
 - Egyptian_cat (21.14%)
 - patas (12.75%)
 - Persian_cat (47.92%)
 - vizsla (24.20%)
 - Egyptian_cat (53.18%)
 - groenendael (13.37%)
 - tabby (17.44%)
 - axolotl (4.48%)
 - grey_fox (29.61%)
 - notebook (10.47%)
 - Egyptian_cat (8.67%)
 - Siamese_cat (45.47%)


In [8]:
from keras.layers import Dense 
from keras import Model

cat_output = Dense(2, activation='softmax')
cat_output = cat_output(model.layers[-2].output) #przedostatnia warstwa
cat_input = model.input
cat_model = Model(inputs=cat_input, outputs=cat_output)


for layer in cat_model.layers[:-1]:
    layer.trainable = False



In [9]:

cat_model.compile(
    loss ='sparse_categorical_crossentropy',
    optimizer = 'adam', 
    metrics = ['accuracy']

)

cat_model.fit(x=data, y=labels, epochs = 20, verbose =2) 

model.summary()


Epoch 1/20
1/1 - 4s - loss: 0.6810 - accuracy: 0.6250 - 4s/epoch - 4s/step
Epoch 2/20
1/1 - 0s - loss: 0.5610 - accuracy: 0.6250 - 317ms/epoch - 317ms/step
Epoch 3/20
1/1 - 0s - loss: 0.5212 - accuracy: 0.7083 - 321ms/epoch - 321ms/step
Epoch 4/20
1/1 - 0s - loss: 0.4728 - accuracy: 0.7083 - 326ms/epoch - 326ms/step
Epoch 5/20
1/1 - 0s - loss: 0.4111 - accuracy: 0.7083 - 343ms/epoch - 343ms/step
Epoch 6/20
1/1 - 0s - loss: 0.3522 - accuracy: 0.9167 - 335ms/epoch - 335ms/step
Epoch 7/20
1/1 - 0s - loss: 0.3068 - accuracy: 0.9583 - 330ms/epoch - 330ms/step
Epoch 8/20
1/1 - 0s - loss: 0.2739 - accuracy: 1.0000 - 325ms/epoch - 325ms/step
Epoch 9/20
1/1 - 0s - loss: 0.2466 - accuracy: 1.0000 - 330ms/epoch - 330ms/step
Epoch 10/20
1/1 - 0s - loss: 0.2201 - accuracy: 1.0000 - 333ms/epoch - 333ms/step
Epoch 11/20
1/1 - 0s - loss: 0.1941 - accuracy: 1.0000 - 325ms/epoch - 325ms/step
Epoch 12/20
1/1 - 0s - loss: 0.1706 - accuracy: 1.0000 - 319ms/epoch - 319ms/step
Epoch 13/20
1/1 - 0s - loss: 0.

In [10]:
predictions = cat_model.predict(data)

1/1 [==============================] - 2s 2s/step


In [11]:
predictions.shape

(24, 2)

In [12]:
np.argmax(predictions, axis =1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1], dtype=int64)

In [18]:
import tensorflow_datasets as tfds
import tensorflow as tf
import keras
from tensorflow.keras import layers

tfds.disable_progress_bar()

train_ds, validation_ds, test_ds = tfds.load(
    "cats_vs_dogs",
    split=["train[:40%]", "train[40%:50%]", "train[50%:60%]"],
    as_supervised=True, 
    batch_size=22
)

size = (224, 224)
train_ds_1 = train_ds.map(lambda x, y: (tf.image.resize(x, size), y))
validation_ds_1 = validation_ds.map(lambda x, y: (tf.image.resize(x, size), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, size), y))

# batch_size = 32
# train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=10)
# validation_ds = validation_ds.cache().batch(batch_size).prefetch(buffer_size=10)
# test_ds = test_ds.cache().batch(batch_size).prefetch(buffer_size=10)



model_m =  keras.applications.MobileNet(
    include_top = False, 
    weights="imagenet", 
    input_shape=(224, 224, 3))

model_m.trainable = False

data_augmentation = keras.Sequential(
    [layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),]
)

x = model_m.layers[-6].output
output = Dense(2, activation='softmax')(x)
model = Model(inputs=model_m.input, outputs=output)
for layer in model.layers[:-5]:
    layer.trainable = False

inputs = keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(x)

x = model_m(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x) 
outputs = keras.layers.Dense(1)(x)
model_m = keras.Model(inputs, outputs)

model_m.summary()


model_m.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

model_m.fit(train_ds_1, epochs=20, validation_data=validation_ds)


Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 sequential_2 (Sequential)   (None, 224, 224, 3)       0         
                                                                 
 rescaling_2 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 mobilenet_1.00_224 (Functio  (None, 7, 7, 1024)       3228864   
 nal)                                                            
                                                                 
 global_average_pooling2d_2   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0   

423/423 [==============================] - 279s 647ms/step - loss: 0.2030 - binary_accuracy: 0.9075 - val_loss: 0.2104 - val_binary_accuracy: 0.9497
Epoch 2/20
423/423 [==============================] - 269s 637ms/step - loss: 0.1270 - binary_accuracy: 0.9399 - val_loss: 0.1655 - val_binary_accuracy: 0.9725
Epoch 3/20
423/423 [==============================] - 258s 611ms/step - loss: 0.1138 - binary_accuracy: 0.9480 - val_loss: 0.1426 - val_binary_accuracy: 0.9751
Epoch 4/20
423/423 [==============================] - 265s 628ms/step - loss: 0.1073 - binary_accuracy: 0.9507 - val_loss: 0.1297 - val_binary_accuracy: 0.9772
Epoch 5/20
423/423 [==============================] - 263s 623ms/step - loss: 0.1077 - binary_accuracy: 0.9502 - val_loss: 0.1249 - val_binary_accuracy: 0.9781
Epoch 6/20
423/423 [==============================] - 258s 610ms/step - loss: 0.1025 - binary_accuracy: 0.9541 - val_loss: 0.1305 - val_binary_accuracy: 0.9802
Epoch 7/20
423/423 [==============================]